# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import dateparser
import unicodedata
import re
import plotly.express as px

# Avoid the dateparse warning
Le package `dateparser` affiche une erreur car lié à une des ses dépendances, on ne peut pas supprimer autrement ce warning que par le bout de code suivant.

Les maintainer de `dateparser` sont au courant du warning, et l'issue est connu, elle ne provient donc pas d'une erreur de notre code:
https://github.com/scrapinghub/dateparser/issues/1013

In [2]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="The localize method is no longer necessary, as this time zone supports the fold attribute",
)

In [3]:
url2021 = "https://fr.wikipedia.org/wiki/Wikipédia:Le_saviez-vous_%3F/Archives"
url2020 = "https://fr.wikipedia.org/wiki/Wikipédia:Le_saviez-vous_%3F/Archives/2020"
url2019 = "https://fr.wikipedia.org/wiki/Wikipédia:Le_saviez-vous_%3F/Archives/2019"
url2018 = "https://fr.wikipedia.org/wiki/Wikipédia:Le_saviez-vous_%3F/Archives/2018"
url2017 = "https://fr.wikipedia.org/wiki/Wikipédia:Le_saviez-vous_%3F/Archives/2017"

# **Scraping the first page**

- Allez sur:
[https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives](https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives)
- Utilisez `requests` pour scraper la page
- Utilisez `BeautifulSoup` pour parser le contenu en un objet `BeautifulSoup`

In [4]:
def get_page(url: str):
    """
    Retreive the wikipedia page and convert it into a BeautifulSoup object
    """
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser")

page2021 = get_page(url2021)

# **Identifying relevant page part and first quote**

- Utilisez l'inspecteur de votre navigateur web pour trouver où se situe le contenu de la page, et les quotes
- Utilisez les propriétés de `BeautifulSoup` pour isoler la partie pertinente
- Isoler la première quote et affichez-la

In [5]:
def get_quotes_on_page(page: BeautifulSoup):
    """
    Get all the quotes on the an html page
    """

    quotes_elements = page.find(class_ = "mw-parser-output").find("ul").find_all("li")
    rows = []

    for quote_element in quotes_elements:
        # Remove the dl tag from the li
        date_element = quote_element.find("dl").extract()

        # unicodedata.normalize("NFKD", child.text) removes the \xa0 character
        quote = unicodedata.normalize("NFKD", quote_element.text)
        quote = quote.replace("\n", "")

        # Add the html elements to our dataframe for post treatements
        row = {
            "quote_element": quote_element,
            "date_element": date_element,
            "quote": quote
        }
        rows.append(row);

        # Reappend the dl tag to the li
        quote_element.append(date_element)

    return rows

rows = get_quotes_on_page(page2021)

rows[0]["quote"]

'Victor Hugo a empêché pendant six ans la représentation en France de Rigoletto (affiche du haut) de Giuseppe Verdi, estimant qu’il s’agissait d’un plagiat d’une de ses œuvres (affiche du bas).'

# **Creating pandas DataFrame**

- Créez un DataFrame pandas afin de récolter les différentes quotes que vous allez parser
- Utilisez les colonnes fournies
- Ajoutez la première quote précédemment extraite comme première entrée de votre DataFrame

```python
# Noms des colonnes du dataframe pandas
columns=['year', 'date', 'quote']
```

In [6]:
def create_quotes_df(data):
    return pd.DataFrame(
        data if data else [],
        columns=[
            "quote_element",
            "date_element",
            "french_date",
            "quote",
            "date",
            "year"
        ]
    )

In [7]:
df_quotes_2021 = create_quotes_df([rows[0]])
df_quotes_2021

,quote_element,date_element,french_date,quote,date,year
0,"[[[<img alt=""Affiche de la première mondiale d...",[[[présente 2 jours en page d'accueil du 01 ja...,NaN,Victor Hugo a empêché pendant six ans la rep...,NaN,NaN


In [8]:
def extract_quote_informations(item):
    """
    Convert the date un french langage into a datetime object
    """
    
    # The regex that removes unwanted informations: https://regex101.com/r/aemAjB/1
    item["french_date"] = re.sub(r"(.*du\s|\sà.*)", "", item.date_element.text);
    item["date"] = dateparser.parse(item.french_date, languages=["fr"])
    item["year"] = item.date.year
    return item

df_quotes_2021 = df_quotes_2021.apply(extract_quote_informations, axis=1)

In [9]:
df_quotes_2021[["quote", "date", "year"]]

,quote,date,year
0,Victor Hugo a empêché pendant six ans la rep...,2021-01-01,2021


# **Parsing 2021**

- L'objectif de cette section est de parser l'année 2021 complète et de l'ajouter au DataFrame
- Les fonction `find`, `rfind`, etc... des `strings` Python peuvent vous être utiles pour identifier les parties pertinentes des quotes
- `dateparser` vous permet de transformer une date sous forme de texte en une date à un format plus standard

In [10]:
# We already parsed the entire df with the previous exercise
df_quotes_2021 = df_quotes_2021.append(rows[1:], ignore_index=True)
df_quotes_2021 = df_quotes_2021.apply(extract_quote_informations, axis=1)

df_quotes_2021[["quote", "date", "year"]]

,quote,date,year
0,Victor Hugo a empêché pendant six ans la rep...,2021-01-01,2021
1,"Au pays des kiwis, la France se nomme Wīwī.",2021-01-01,2021
2,"Réalisés le même jour, La Rue Mosnier aux d...",2021-01-01,2021
3,L’astéroïde (24601) Valjean est ainsi nommé...,2021-01-02,2021
4,"D’abord hôtel particulier, désormais réside...",2021-01-03,2021
...,...,...,...
845,"En 1957, Dax remporte le Challenge Yves du Man...",2021-12-05,2021
846,"L’« étoile de Tabby » (illustration), située...",2021-12-06,2021
847,"Après la réunification allemande, la capital...",2021-12-06,2021
848,"Inspirée de la ligne Maginot, la ligne Chauvi...",2021-12-07,2021


# **Parsing 2017 to 2020**

- L'objectif de cette section est de parser les années 2017 à 2020 et de les ajouter au DataFrame
- Les fonction `find`, `rfind`, etc... des `strings` Python peuvent vous être utiles pour identifier les parties pertinentes des quotes
- `dateparser` vous permet de transformer une date sous forme de texte en une date à un format plus standard

In [11]:
urls = [url2017, url2018, url2019, url2020]

df_quotes_2017_2021 = df_quotes_2021.copy()

# Parse all the pages from 2017 to 2020 and append it in a dataframe that already contains the quotes from 2021
for url in urls:
    page = get_page(url)
    quotes = get_quotes_on_page(page)
    df_quotes_2017_2021 = df_quotes_2017_2021.append(quotes, ignore_index=True)

# Convert the french date into a datetime object for each rows
df_quotes_2017_2021 = df_quotes_2017_2021.apply(extract_quote_informations, axis=1)

# Remove the quotes that were published before 2017 (wikipedia page error)
df_quotes_2017_2021 = df_quotes_2017_2021[df_quotes_2017_2021.year >= 2017]

# Sort the dataframe by dates
df_quotes_2017_2021.sort_values(by=["date"], inplace=True)

# Reset the index and removes the old one
df_quotes_2017_2021.reset_index(inplace=True)
df_quotes_2017_2021.drop("index", axis=1, inplace=True)

df_quotes_2017_2021[["quote", "date", "year"]]
    

,quote,date,year
0,Aucun État ayant un programme autonome de vol...,2017-01-01,2017
1,"Le 1er janvier 1977, la Belgique a divisé par...",2017-01-01,2017
2,Le fruit de la pomme de terre (dessin) est tox...,2017-01-02,2017
3,Une loi adoptée sous le Consulat a autorisé ...,2017-01-02,2017
4,Le Paléolithique recouvre environ 95 % de la ...,2017-01-03,2017
...,...,...,...
4969,"En 1957, Dax remporte le Challenge Yves du Man...",2021-12-05,2021
4970,"L’« étoile de Tabby » (illustration), située...",2021-12-06,2021
4971,"Après la réunification allemande, la capital...",2021-12-06,2021
4972,"Inspirée de la ligne Maginot, la ligne Chauvi...",2021-12-07,2021


# **df Transformations**

- Supprimez la colonne `'year'` du DataFrame
- Importez de la colonne `'date'` des informations permettant de générer une colonne `'year'` et `'month'`

In [12]:
df_quotes_2017_2021.drop("year", axis=1, inplace=True)
df_quotes_2017_2021["month"] = df_quotes_2017_2021.date.dt.month
df_quotes_2017_2021["year"] = df_quotes_2017_2021.date.dt.year

df_quotes_2017_2021[["quote", "date", "year"]]

,quote,date,year
0,Aucun État ayant un programme autonome de vol...,2017-01-01,2017
1,"Le 1er janvier 1977, la Belgique a divisé par...",2017-01-01,2017
2,Le fruit de la pomme de terre (dessin) est tox...,2017-01-02,2017
3,Une loi adoptée sous le Consulat a autorisé ...,2017-01-02,2017
4,Le Paléolithique recouvre environ 95 % de la ...,2017-01-03,2017
...,...,...,...
4969,"En 1957, Dax remporte le Challenge Yves du Man...",2021-12-05,2021
4970,"L’« étoile de Tabby » (illustration), située...",2021-12-06,2021
4971,"Après la réunification allemande, la capital...",2021-12-06,2021
4972,"Inspirée de la ligne Maginot, la ligne Chauvi...",2021-12-07,2021


# **Exporting DataFrame to csv**

- Exporter votre DataFrame avec le nom donné
- Utilisez `';'` comme séparateur et n'exporter pas l'index

In [13]:
df_quotes_2017_2021.drop(["quote_element", "date_element", "french_date"], axis=1, inplace=True)

df_quotes_2017_2021.to_csv("Wikiquotes.csv", sep=";", index=False)

df_quotes_2017_2021

,quote,date,month,year
0,Aucun État ayant un programme autonome de vol...,2017-01-01,1,2017
1,"Le 1er janvier 1977, la Belgique a divisé par...",2017-01-01,1,2017
2,Le fruit de la pomme de terre (dessin) est tox...,2017-01-02,1,2017
3,Une loi adoptée sous le Consulat a autorisé ...,2017-01-02,1,2017
4,Le Paléolithique recouvre environ 95 % de la ...,2017-01-03,1,2017
...,...,...,...,...
4969,"En 1957, Dax remporte le Challenge Yves du Man...",2021-12-05,12,2021
4970,"L’« étoile de Tabby » (illustration), située...",2021-12-06,12,2021
4971,"Après la réunification allemande, la capital...",2021-12-06,12,2021
4972,"Inspirée de la ligne Maginot, la ligne Chauvi...",2021-12-07,12,2021


# **Statistics**
Utilisez les fonctions ou propriétés de votre choix afin de répondre aux questions suivantes

### Y a-t-il des lignes pour lesquelles certains champs sont vides ?
Si oui, combien ?

In [14]:
df_quotes_2017_2021.isna()["quote"].value_counts()

False    4974
Name: quote, dtype: int64

Non, il y'a aucune quote vide (Toutes les valeurs qui disent si quote est vide sont à False)

### Quelle année contient le moins de quotes, et combien?

In [15]:
quotes_count = df_quotes_2017_2021.groupby("year").count()["quote"]
quotes_count.idxmin()

2019

### Quelle est la quote la plus longue?
(En terme de nombre de caractères)

In [16]:
df_quotes_2017_2021.iloc[df_quotes_2017_2021["quote"].str.len().idxmax()]["quote"]

'Le prince de Talleyrand-Périgord fut évêque, député, président de l’Assemblée constituante de 1789, spéculateur et armateur, ministre de la marine et des colonies sous le Directoire, ministre des relations extérieures sous ce même régime puis sous l’Empire, grand chambellan de France sous deux régimes, prince-électeur impérial, président du gouvernement provisoire royaliste de 1814, ministre des Affaires étrangères sous la première Restauration, chef du gouvernement sous la seconde, trois fois ambassadeur, et enfin maire et conseiller général de l’Indre (caricature).'

### Quelle est la quote la plus courte?
(En terme de nombre de caractères)

In [17]:
df_quotes_2017_2021.iloc[df_quotes_2017_2021["quote"].str.len().idxmin()]["quote"]

'Film en est un.'

### Combien de quotes contiennent `'Victor Hugo'` ?

In [18]:
df_quotes_2017_2021["quote"].str.contains("Victor Hugo").value_counts()[True]

16

### J'ai entendu dire qu'il y avait une quote qui parlait de pokémon, laquelle est-ce ? (Affichez la quote complète)

In [19]:
# Remove the accents of the quote and transform it into lowercase
df_quotes_2017_2021["normalized_quote"] = df_quotes_2017_2021.quote.str.encode("ascii", errors="ignore").str.decode("utf-8").str.lower()

# Add a column that tells if the quote contains pokemon
df_quotes_2017_2021["contains_pokemon"] = df_quotes_2017_2021.normalized_quote.str.contains("pokemon")

df_quotes_2017_2021[df_quotes_2017_2021.contains_pokemon == True].iloc[0]["quote"]

'En mars 2017, un jeune Russe a été condamné à 3 ans et demi de colonie pénitentiaire pour avoir joué à Pokémon Go dans une église.'

## Afficher le nombre de quotes moyennes par mois

In [34]:
df_quotes_2017_2021 \
    .groupby(["month", "year"]) \
    .count() \
    .groupby("month") \
    .mean()[["quote"]]

,quote
month,
1,74.6
2,66.8
3,78.0
4,86.8
5,99.6
6,83.0
7,86.0
8,85.8
9,81.6


# **Visualisation**

Affichez un graphe de votre choix pour visualiser les données, que vous jugez pertinent (par exemple un barplot, scatterplot, ou autre)

## Affichage du nombre de quote par année

In [21]:
df_year_quotes_count = df_quotes_2017_2021.groupby("year").count()[["quote"]].reset_index()

px.bar(
    df_year_quotes_count,
    x="year",
    y="quote",
    labels={
        "quote": "Nombre de quote",
        "year": "Année"
    }
)

## Affichage du nombre de quote par mois depuis 2017 à 2021

In [22]:
df_month_quotes_count = df_quotes_2017_2021.groupby(["year", "month"]).count()["quote"].reset_index()
df_month_quotes_count["month_year"] = df_month_quotes_count.apply(lambda x: datetime(x.year, x.month, 1), axis=1)
df_month_quotes_count.sort_values("month_year")

px.bar(
    df_month_quotes_count,
    x="month_year",
    y="quote",
    labels={
        "quote": "Nombre de quote",
        "month_year": "Mois et année"
    }
)

## Nombre de quote sous forme de boxplot

In [23]:
px.box(
    df_year_quotes_count,
    y="quote",
    points="all",
    labels={
        "quote": "Nombre de quote"
    }
)